# Calling the LangGraph Multi-Modal Research Agent API

This notebook demonstrates how to call the deployed LangGraph research agent API.

In [ ]:
# Install necessary libraries
!pip install requests -q

In [ ]:
import requests
import json
import os

## Configuration

**IMPORTANT:** The `LANGGRAPH_API_INVOKE_URL` below is set to the public URL you provided. If the application is not deployed there or the invoke path is different (e.g., `/research_agent/invoke` instead of just `/invoke`), you'll need to adjust it.

In [ ]:
# Configuration 
LANGGRAPH_API_INVOKE_URL = "https://multi-modal-researcher-675059836631.us-central1.run.app/invoke"

## Helper Function to Call the Agent

In [ ]:
def call_research_agent(payload):
    """Sends a request to the LangGraph agent and returns the response."""
    headers = {
        "Content-Type": "application/json"
    }
    print(f"Sending request to: {LANGGRAPH_API_INVOKE_URL}")
    print(f"Payload:\n{json.dumps(payload, indent=2)}")
    
    try:
        response = requests.post(LANGGRAPH_API_INVOKE_URL, headers=headers, json=payload, timeout=600) # 10 minute timeout
        response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
        
        print("\n--- Full API Response (raw) ---")
        response_json = response.json()
        print(json.dumps(response_json, indent=2))
        
        # The LangGraph server typically nests the actual graph output under an 'output' key
        if isinstance(response_json, dict) and "output" in response_json:
            return response_json["output"]
        else:
            print("Warning: 'output' key not found in response JSON or response is not a dict, returning full JSON as is.")
            return response_json
            
    except requests.exceptions.HTTPError as http_err:
        print(f"\nHTTP error occurred: {http_err}")
        print(f"Response content: {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"\nConnection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"\nTimeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"\nAn unexpected request error occurred: {req_err}")
    except json.JSONDecodeError as json_err:
        print(f"\nFailed to decode JSON response: {json_err}")
        print(f"Response content probably not JSON: {response.text}")
    return None

## Test Case 1: "Topic Only" Research

In [ ]:
print("\n--- Test Case 1: Topic Only Research ---")
topic_only_input_payload = {
    "input": {
        "topic": "The history and evolution of functional programming languages",
        "research_approach": "Topic Only",
        "company_name": None,
        "title_areas": None,
        "video_url": None, 
        "create_podcast": False
    }
}

results_topic_only = call_research_agent(topic_only_input_payload)

if results_topic_only:
    print("\n--- Results for Topic Only Research ---")
    print(f"Report Snippet: {str(results_topic_only.get('report', 'N/A'))[:500]}...")
    if results_topic_only.get('podcast_url'):
        print(f"Podcast URL: {results_topic_only.get('podcast_url')}")
else:
    print("Failed to get results for Topic Only research.")

## Test Case 2: "Topic Company Leads" Research

**Note:** For this to fully work, the deployed LangGraph agent's environment must have the necessary API keys set up:
- `GEMINI_API_KEY` (for Gemini calls)
- `GOOGLE_API_KEY_FOR_CSE` (for CSE LinkedIn search)
- `GOOGLE_CSE_ID` (for CSE LinkedIn search)
- `GCS_BUCKET_NAME` and `GOOGLE_APPLICATION_CREDENTIALS` (if podcast/report GCS upload is expected)

In [ ]:
print("\n\n--- Test Case 2: Topic Company Leads Research ---")
topic_company_leads_input_payload = {
    "input": {
        "topic": "The role of Vector Databases in Generative AI applications",
        "research_approach": "Topic Company Leads",
        "company_name": "Pinecone",
        "title_areas": ["Solutions Architect", "Developer Advocate", "Head of Engineering"],
        "video_url": None,
        "create_podcast": False
    }
}

results_company_leads = call_research_agent(topic_company_leads_input_payload)

if results_company_leads:
    print("\n--- Results for Topic Company Leads Research ---")
    print(f"Report Snippet: {str(results_company_leads.get('report', 'N/A'))[:500]}...")
    
    print("\nIdentified Leads (from Gemini):")
    gemini_leads = results_company_leads.get('identified_leads')
    if gemini_leads:
        for i, lead in enumerate(gemini_leads[:2]): # Print first 2 leads
            print(f"  Lead {i+1}:")
            print(f"    Name: {lead.get('lead_name')}")
            print(f"    Title: {lead.get('lead_title')}")
            print(f"    Department: {lead.get('lead_department')}")
            print(f"    LinkedIn: {lead.get('linkedin_url')}")
            print(f"    Relevance: {lead.get('summary_of_relevance')}")
            if lead.get('named_buyers'):
                print("    Named Buyers:")
                for buyer in lead['named_buyers']:
                    print(f"      - {buyer.get('buyer_name')} ({buyer.get('buyer_title')}): {buyer.get('buyer_rationale')}")
    else:
        print("  No leads identified by Gemini or field not present.")
        
    print("\nLinkedIn Contacts (from CSE):")
    cse_contacts = results_company_leads.get('linkedin_cse_contacts')
    if cse_contacts:
        for i, contact in enumerate(cse_contacts[:3]): # Print first 3 CSE contacts
            print(f"  Contact {i+1}:")
            print(f"    Title: {contact.get('title')}")
            print(f"    Link: {contact.get('link')}")
            print(f"    Snippet: {contact.get('snippet')}")
    else:
        print("  No LinkedIn contacts found via CSE or field not present.")

    if results_company_leads.get('podcast_url'):
        print(f"\nPodcast URL: {results_company_leads.get('podcast_url')}")
else:
    print("Failed to get results for Topic Company Leads research.")

## Conclusion

This notebook provides a template for interacting with the LangGraph research agent. Remember to:
1. Ensure your LangGraph agent is deployed and accessible at the `LANGGRAPH_API_INVOKE_URL` specified.
2. Verify the invoke path (e.g., `/invoke` or `/<graph_id>/invoke`).
3. Ensure all necessary API keys (`GEMINI_API_KEY`, `GOOGLE_API_KEY_FOR_CSE`, `GOOGLE_CSE_ID`) are correctly configured in the environment where your LangGraph agent is running for full functionality.